In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import re
import time

import pandas as pd
import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm

<IPython.core.display.Javascript object>

In [5]:
def browser(url):
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    driver = webdriver.Chrome(options=option)
    driver.get(url)
    return driver


def get_html_full(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_full_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_from_a_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[2]/of-xpa-switch-match/of-match-events/div/button"
    )
    el.click()
    time.sleep(2)
    page = driver.page_source
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[3]/div[1]/div/of-xpa-switch-match/of-match-lineup-v2/section/nav/ul/li[2]/button"
    )
    el.click()
    time.sleep(2)
    second_lineup = driver.page_source
    driver.close()
    return BeautifulSoup(page), BeautifulSoup(second_lineup)



def format_field(string):
    string = str(string)
    string = string.lower()
    string = string.strip()
    string = string.replace("  ", " ")
    string = string.replace(" ", "_")
    return string


def get_info_from_event(event):
    aux_event = {}
    aux_event["event_team"] = event["class"][-1][-4:]
    aux_event["event_time"] = event.find(
        "div", class_="match-events__item-timeline"
    ).text

    if aux_event["event_time"] == " PK ":
        aux_event["event_time"] = False
        aux_event["event_type"] = "PK"
        aux_event["event_result"] = event.find("img", class_="of-image__img")["alt"]
        aux_event["event_player"] = (
            event.find("div", class_="match-events__text").find("p").text
        )
    else:
        try:
            aux_event["event_type"] = event.find(class_="match-events__icon")[
                "aria-label"
            ]
        except:
            aux_event["event_type"] = event.find("img", class_="of-image__img")["alt"]
        try:
            for i, text in enumerate(
                event.find("div", class_="match-events__text").find_all("p")
            ):
                aux_event["action_player_" + str(i + 1)] = text.text
        except:
            aux_event["action_player_1"] = event.find(
                "p", class_="match-events__text"
            ).text

    return aux_event


def get_info_from_match(page, second_lineup):
    aux_dict = {}

    ## NAMES
    aux_dict["team_name_home"] = page.find(
        "span", class_="match-score-team__name--home"
    ).text
    aux_dict["team_name_away"] = page.find(
        "span", class_="match-score-team__name--away"
    ).text

    ## GOAL
    scores = page.find("p", class_="match-score-scores").find_all("span")
    aux_dict["team_home_score"] = scores[0].text
    aux_dict["team_away_score"] = scores[2].text

    ## STATISTICS
    description = page.find_all("div", class_="match-stats__stat-description")
    for d in description:
        field = format_field(d.find_all("p")[1].text)
        aux_dict[field + "_home"] = d.find_all("p")[0].text
        aux_dict[field + "_away"] = d.find_all("p")[2].text

    ## EVENTS
    events = page.find("ul", class_="match-events__list").find_all(
        "li", class_="match-events__item"
    )
    aux_dict["events_list"] = [get_info_from_event(event) for event in events]

    aux_dict["lineup_home"] = get_lineups(page)
    aux_dict["lineup_away"] = get_lineups(second_lineup)

    return aux_dict


def get_lineups(page):
    lineup = []
    for player in page.find_all("div", class_="match-lineup__player"):
        lineup.append(
            {
                "Player_Name": player.find(
                    "p", class_="match-lineup__player-name"
                ).text,
                "Player_Number": format_field(
                    player.find("span", class_="match-lineup__jersey-number").text
                ),
            }
        )
    return lineup



def get_penaltis(match):
    regex_rule = r"\((\d+)\)"
    aux = match.find_all(
        "span", class_="title-7-bold simple-match-card-team__score"
    )
    aux = [
        re.search(regex_rule, i.text)[0].replace("(", "").replace(")", "") for i in aux
    ]
    return aux


def get_all_matches(html_full):
    aux_dict = []
    for match in html_full.find_all("a", class_="match-card", href=True):
        aux_dict_2 = {}
        aux_dict_2["link"] = "https://onefootball.com" + match["href"]
        aux_dict_2["stage"] = match.find_previous(
            "p", class_="label simple-match-cards-list__title-deprecated"
        ).text
        try:
            aux_dict_2["date"] = match.find("time").text
            aux_dict_2["pens"] = False
            aux_dict_2["pens_home_score"] = False
            aux_dict_2["pens_away_score"] = False
        except:
            aux_dict_2["date"] = False
            aux_dict_2["pens"] = (
                match.find(
                    "span",
                    class_="title-8-medium simple-match-card__info-message simple-match-card__info-message--secondary",
                ).text
                == "(Pens)"
            )
            home, away = get_penaltis(match)
            aux_dict_2["pens_home_score"] = home
            aux_dict_2["pens_away_score"] = away

        aux_dict.append(aux_dict_2)
    return aux_dict


<IPython.core.display.Javascript object>

## Competition

In [4]:
url = "https://onefootball.com/en/competition/olympic-games-women-54/results"  ## Competition link
page = get_html_full(url)
matchs = get_all_matches(page)
matchs[0]

{'link': 'https://onefootball.com/en/match/2210723',
 'stage': ' Final ',
 'date': False,
 'pens': True,
 'pens_home_score': '2',
 'pens_away_score': '3'}

<IPython.core.display.Javascript object>

In [6]:
%%time

def merge_information(match):
    match_info = get_info_from_match(*get_html_from_a_match(match["link"]))
    return{**match, **match_info}
    
from joblib import Parallel, delayed
aux_list = Parallel(n_jobs=6, verbose=10)(delayed(merge_information)(match) for match in matchs)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   10.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   10.6s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:   30.2s
[Parallel(n_jobs=6)]: Done  18 out of  26 | elapsed:   32.4s remaining:   14.3s
[Parallel(n_jobs=6)]: Done  21 out of  26 | elapsed:   39.9s remaining:    9.4s
[Parallel(n_jobs=6)]: Done  24 out of  26 | elapsed:   40.7s remaining:    3.3s


Wall time: 48.5 s


[Parallel(n_jobs=6)]: Done  26 out of  26 | elapsed:   48.3s finished


<IPython.core.display.Javascript object>

In [9]:
df = pd.DataFrame(aux_list).drop("link", axis=1)
print(df.shape)
df

(26, 20)


,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,False,True,2,3,Sweden Women,Canada Women,1,1,52%,48%,30,14,6,4,47%,53%,"[{'event_team': 'away', 'event_time': ' 27' ',...",[],[]
1,Match for third place,05.08.2021,False,False,False,Australia Women,USA Women,3,4,52%,48%,13,19,4,9,55%,45%,"[{'event_team': 'away', 'event_time': ' 8' ', ...",[],[]
2,Semi-finals,02.08.2021,False,False,False,Australia Women,Sweden Women,0,1,58%,42%,14,10,3,4,58%,43%,"[{'event_team': 'away', 'event_time': ' 46' ',...",[],[]
3,Semi-finals,02.08.2021,False,False,False,USA Women,Canada Women,0,1,56%,44%,17,5,6,2,48%,52%,"[{'event_team': 'home', 'event_time': ' 30' ',...",[],[]
4,Quarter-finals,False,True,2,4,Netherlands Women,USA Women,2,2,49%,51%,21,16,7,4,47%,53%,"[{'event_team': 'home', 'event_time': ' 18' ',...",[],[]
5,Quarter-finals,30.07.2021,False,False,False,Sweden Women,Japan Women,3,1,33%,67%,16,13,6,7,46%,54%,"[{'event_team': 'home', 'event_time': ' 7' ', ...",[],[]
6,Quarter-finals,30.07.2021,False,False,False,Great Britain Women,Australia Women,3,4,55%,45%,21,14,10,5,53%,47%,"[{'event_team': 'away', 'event_time': ' 26' ',...",[],[]
7,Quarter-finals,False,True,4,3,Canada Women,Brazil Women,0,0,46%,54%,13,16,2,6,49%,51%,"[{'event_team': 'away', 'event_time': ' 58' ',...",[],[]
8,Round 3,27.07.2021,False,False,False,Netherlands Women,China Women,8,2,61%,39%,20,11,13,2,52%,48%,"[{'event_team': 'home', 'event_time': ' 12' ',...",[],[]
9,Round 3,27.07.2021,False,False,False,Brazil Women,Zambia Women,1,0,73%,27%,17,5,7,1,54%,46%,"[{'event_team': 'away', 'event_time': ' 13' ',...",[],[]


<IPython.core.display.Javascript object>

## Manual adjustment

In [8]:
df[df.date == False]

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,False,True,2,3,Sweden Women,Canada Women,1,1,52%,48%,30,14,6,4,47%,53%,"[{'event_team': 'away', 'event_time': ' 27' ',...",[],[]
4,Quarter-finals,False,True,2,4,Netherlands Women,USA Women,2,2,49%,51%,21,16,7,4,47%,53%,"[{'event_team': 'home', 'event_time': ' 18' ',...",[],[]
7,Quarter-finals,False,True,4,3,Canada Women,Brazil Women,0,0,46%,54%,13,16,2,6,49%,51%,"[{'event_team': 'away', 'event_time': ' 58' ',...",[],[]


<IPython.core.display.Javascript object>

In [ ]:
df.date.iloc[0] = "06.08.2021"  ## Sweden Women X Canada Women
df.date.iloc[4] = "06.07.2021"  ## Netherlands Women X USA Women
df.date.iloc[7] = "02.07.2021"  ## Canada Women X Brazil Women

In [ ]:
df.to_csv("fotball_olympics_games_woman_2020/2021.csv", index=False)